## Summary

Repo: https://github.com/pgahq/instructor-groq-openai-llm-examples

This notebook shows how to use Instructor with nested models to make an analysis (of a new business idea). At the end, it'll even suggest a new, improved version of the business idea.

Note: this notebook assumes you're using Google Colab. You can safely edit / play here. Or go to `File` -> `Save a copy in Google Drive` to make your own version.



In [1]:
!pip install --quiet instructor groq openai


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


On the left, click the key and set two secrets with your keys. Be sure to enable "Notebook access" for them. This is how Google Colab works...you're not sharing your keys with anyone.

OPENAI_API_KEY - get a key from https://platform.openai.com/api-keys

GROQ_API_KEY - get a key from https://console.groq.com/keys

In [2]:
import instructor
import openai
import groq
from pydantic import BaseModel, Field
from typing import Optional, List
import os

try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = '' or userdata.get('OPENAI_API_KEY') # or put your key in the '' on this line
    os.environ['GROQ_API_KEY'] = '' or userdata.get('GROQ_API_KEY')
except Exception as e:
    # print(e)
    pass

if not os.environ.get('OPENAI_API_KEY') or not os.environ.get('GROQ_API_KEY'):
    raise ValueError("Both OPENAI_API_KEY and GROQ_API_KEY environment variables must be set and non-empty. Read the text in the notebook (above this block) for more info.")

In [3]:
company = "Luigi's Murano Glass Company"
company_description = """
Luigi's Murano Glass Company is a leading manufacturer of high-quality, affordable, and stylish glass products. With a focus on innovative design and durability, Luigi's Murano Glass Company offers a wide range of glass solutions to meet the needs of both residential and commercial customers. With over 20 years of manufacturing experience, Luigi's Murano Glass Company has a proven track record of delivering high-quality glass products at unbeatable prices. Our factory store is located in Murano, Italy.
"""

new_business_idea = """
New Business Idea: Partnering with US Interior Architecture Firms to Distribute Official Murano Glass Sinks

Objectives:
1. Establish Strategic Partnerships:
   - Collaborate with top US interior architecture firms to integrate our Murano glass sinks into their design projects.
   - Develop co-branded marketing materials to highlight the partnership and the unique value of Murano glass sinks.
   - Offer exclusive deals and discounts to partner firms to incentivize the use of our products in their projects.

2. Increase Market Penetration:
   - Expand our market reach by tapping into the client base of established interior architecture firms.
   - Utilize the firms' existing relationships with high-end residential and commercial clients to introduce our products.
   - Leverage the firms' expertise to tailor our product offerings to meet the specific needs and preferences of their clients.

3. Enhance Brand Visibility:
   - Elevate the brand visibility of Luigi's Murano Glass Company through high-profile design projects.
   - Feature our Murano glass sinks in prominent design publications and online platforms.
   - Participate in industry events and trade shows alongside our partner firms to showcase our products.

4. Drive Revenue Growth:
   - Generate additional revenue through increased sales of Murano glass sinks to architecture firms and their clients.
   - Offer premium pricing for custom-designed sinks that cater to the unique requirements of high-end projects.
   - Explore opportunities for bulk sales and long-term contracts with partner firms.

Execution Plan:
1. Identify Potential Partners:
   - Research and shortlist top interior architecture firms in key US markets.
   - Reach out to potential partners with a compelling proposal highlighting the benefits of collaboration.

2. Develop Partnership Agreements:
   - Draft and negotiate partnership agreements that outline the terms of collaboration, including pricing, exclusivity, and marketing commitments.
   - Ensure that the agreements align with our business objectives and provide mutual benefits.

3. Create Marketing and Sales Materials:
   - Develop co-branded marketing materials, including brochures, catalogs, and online content, to promote the partnership.
   - Train our sales team to effectively communicate the value proposition of our Murano glass sinks to architecture firms and their clients.

4. Launch and Promote the Partnership:
   - Announce the partnership through press releases, social media, and industry publications.
   - Host launch events and webinars to introduce the partnership to the design community and potential clients.

5. Monitor and Evaluate Performance:
   - Track the performance of the partnership through sales data, client feedback, and market analysis.
   - Regularly review and adjust the partnership strategy to ensure it meets our business objectives and maximizes value for both parties.

Market Analysis:
- Market Need Score: Highlight the growing demand for unique and high-quality design elements in interior architecture.
- Market Size Score: Emphasize the substantial market size of the US interior architecture industry and the potential for growth.
- Competitive Landscape Score: Address the competitive landscape and the need for differentiation through exclusive partnerships and unique product offerings.

Overall Strategy:
- Ensure a well-rounded and promising strategy with significant potential benefits through strategic partnerships.
- Address challenges such as competitive landscape, partnership management, and market adaptation.
- Focus on enhancing brand visibility, increasing market penetration, and driving revenue growth through collaboration with interior architecture firms.
- Develop contingency plans to mitigate potential risks and ensure smooth execution of the partnership strategy.

"""

In [42]:

class Score(BaseModel):
    score: float = Field(description=f"Score for the strategy evaluation.")
    explanation: str = Field(description=f"Explanation / justification for the score.")

class Objective(BaseModel):
    name: str = Field(description=f"Name of the strategic objective derived from the strategy document.")
    description: str = Field(description=f"Description of the strategic objective derived from the strategy document.")
    relevance_to_objective_score: Score = Field(description=f"Rate and justify the relevance of this objective on a 5-point scale, from 1 (least relevant) to 5 (most relevant).")

class MarketAnalysis(BaseModel):
    market_need_score: Score = Field(description=f"Rate and justify the market need for the business idea on a 5-point scale, from 1 (low need) to 5 (high need).")
    market_size_score: Score = Field(description=f"Rate and justify the market size for the business idea on a 5-point scale, from 1 (small market) to 5 (large market).")
    competitive_landscape_score: Score = Field(description=f"Rate and justify the competitive landscape for the business idea on a 5-point scale, from 1 (high competition) to 5 (low competition).")

class FinancialAnalysis(BaseModel):
    revenue_projection_score: Score = Field(description=f"Rate the revenue projection on a 5-point scale, from 1 (low projection) to 5 (high projection).")
    cost_analysis_score: Score = Field(description=f"Rate the cost analysis on a 5-point scale, from 1 (high costs) to 5 (low costs).")
    profitability_score: Score = Field(description=f"Rate the profitability on a 5-point scale, from 1 (low profitability) to 5 (high profitability).")
    funding_requirements_score: Score = Field(description=f"Rate the funding requirements on a 5-point scale, from 1 (high funding needs) to 5 (low funding needs).")

class OperationalFeasibility(BaseModel):
    resource_availability_score: Score = Field(description=f"Rate the availability of necessary resources on a 5-point scale, from 1 (low availability) to 5 (high availability).")
    operational_requirements_score: Score = Field(description=f"Rate the operational requirements on a 5-point scale, from 1 (high complexity) to 5 (low complexity).")
    scalability_score: Score = Field(description=f"Rate the scalability on a 5-point scale, from 1 (low scalability) to 5 (high scalability).")

class RiskAnalysis(BaseModel):
    market_risk_score: Score = Field(description=f"Rate the market risk on a 5-point scale, from 1 (high risk) to 5 (low risk).")
    operational_risk_score: Score = Field(description=f"Rate the operational risk on a 5-point scale, from 1 (high risk) to 5 (low risk).")
    financial_risk_score: Score = Field(description=f"Rate the financial risk on a 5-point scale, from 1 (high risk) to 5 (low risk).")
    legal_regulatory_risk_score: Score = Field(description=f"Rate the legal and regulatory risk on a 5-point scale, from 1 (high risk) to 5 (low risk).")

class ImpactAnalysis(BaseModel):
    client_impact_score: Score = Field(description=f"Rate the impact on clients on a 5-point scale, from 1 (low impact) to 5 (high impact).")
    business_growth_score: Score = Field(description=f"Rate the impact on the growth of the business on a 5-point scale, from 1 (low impact) to 5 (high impact).")
    brand_impact_score: Score = Field(description=f"Rate the impact on the company's brand on a 5-point scale, from 1 (low impact) to 5 (high impact).")
    sustainability_score: Score = Field(description=f"Rate the sustainability impact on a 5-point scale, from 1 (low impact) to 5 (high impact).")

class StrategyEvaluation(BaseModel):
    """
    Critical analysis of a new business idea.
    """
    title: str = Field(description=f"Title of the strategy document being evaluated.")
    objectives: List[Objective] = Field(description=f"List of strategic objectives and their respective relevance scores and justifications.")
    market_analysis: MarketAnalysis = Field(description=f"Market analysis scores and detailed justifications, including market need, market size, and competitive landscape.")
    financial_analysis: FinancialAnalysis = Field(description=f"Financial analysis scores and detailed justifications, including revenue projections, cost analysis, profitability, and funding requirements.")
    operational_feasibility: OperationalFeasibility = Field(description=f"Operational feasibility scores and detailed justifications, including resource availability, operational requirements, and scalability.")
    risk_analysis: RiskAnalysis = Field(description=f"Risk analysis scores and detailed justifications, including market risk, operational risk, financial risk, and legal/regulatory risk.")
    impact_analysis: ImpactAnalysis = Field(description=f"Impact analysis scores and detailed justifications, including client impact, business growth impact, and brand impact.")
    # overall_score: float = Field(description=f"Overall score (an average of all scores, rounded to one decimal place), including strategic objectives, market analysis, financial analysis, operational feasibility, risk analysis, and impact analysis.")
    improved_new_business_idea_text: str = Field(description=f"Complete revised proposal text to maximize the overall score. The only thing that needs to stay the same is the title. Use markdown. No commentary or other text...just the revised business idea text.")


### Note: this takes 10-30 seconds

In [45]:
inference_provider = "openai"   # "openai" or "groq"
client = instructor.from_openai(openai.OpenAI()) if inference_provider == "openai" else instructor.from_groq(groq.Groq())

result = client.chat.completions.create(
    model="llama-3.1-70b-versatile" if inference_provider == "groq" else "gpt-4o-mini",
    response_model=StrategyEvaluation,
    temperature=0.0,
    messages=[
      {"role": "user", "content": "Company: " + company},
      {"role": "user", "content": "Company Description: " + company_description},
      {"role": "user", "content": "New Business Idea: " + new_business_idea}
      ]
    )

print(result.model_dump_json(indent=4))

{
    "title": "Partnering with US Interior Architecture Firms to Distribute Official Murano Glass Sinks",
    "objectives": [
        {
            "name": "Establish Strategic Partnerships",
            "description": "Collaborate with top US interior architecture firms to integrate our Murano glass sinks into their design projects.",
            "relevance_to_objective_score": {
                "score": 5.0,
                "explanation": "Strategic partnerships with interior architecture firms are crucial for gaining access to high-end clients and enhancing product visibility."
            }
        },
        {
            "name": "Increase Market Penetration",
            "description": "Expand our market reach by tapping into the client base of established interior architecture firms.",
            "relevance_to_objective_score": {
                "score": 5.0,
                "explanation": "Leveraging existing relationships of architecture firms will significantly enhance mark